In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/crimes.csv', parse_dates=["Date Rptd","DATE OCC"], low_memory=False)

In [ ]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION,hour
0,221412410,2022-06-15,2020-11-12,1700,Pacific,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0,NaN,NaN,NaN,Invest Cont,13600 MARINA POINT DR,17
1,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV,11
2,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST,16
3,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT,12
4,231207725,2023-02-27,2020-01-27,635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV,6


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247988 entries, 0 to 247987
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DR_NO         247988 non-null  int64         
 1   Date Rptd     247988 non-null  datetime64[ns]
 2   DATE OCC      247988 non-null  datetime64[ns]
 3   TIME OCC      247988 non-null  int64         
 4   AREA NAME     247988 non-null  object        
 5   Crm Cd Desc   247988 non-null  object        
 6   Vict Age      247988 non-null  int64         
 7   Vict Sex      215740 non-null  object        
 8   Vict Descent  215739 non-null  object        
 9   Weapon Desc   80087 non-null   object        
 10  Status Desc   247988 non-null  object        
 11  LOCATION      247988 non-null  object        
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage: 22.7+ MB


In [ ]:
time_occ = pd.to_numeric(df["TIME OCC"], errors="coerce").fillna(0).astype(int)

In [ ]:
#time_occ

In [ ]:
hours = (time_occ // 100).clip(0, 23)

In [ ]:
df["hour"] = hours

In [ ]:
peak_crime_hour = int(df["hour"].value_counts().idxmax())

In [ ]:
peak_crime_hour

12

In [ ]:
night_mask = (df["hour"] >= 22) | (df["hour"] <= 3)

In [ ]:
night_df = df.loc[night_mask].copy()

In [ ]:
peak_night_crime_location = (
    night_df["AREA NAME"].value_counts().idxmax()
    if not night_df.empty else np.nan)

In [ ]:
peak_night_crime_location

'Central'

In [ ]:
peak_night_crime_location

'Central'

In [ ]:
counts = night_df["AREA NAME"].value_counts()
top_count = counts.max()

In [ ]:
top_count

4091

In [ ]:
valid_age = pd.to_numeric(df["Vict Age"], errors="coerce")
valid_age = valid_age.where(valid_age > 0)

In [ ]:
valid_age

,Vict Age
0,NaN
1,27.0
2,60.0
3,28.0
4,37.0
...,...
247983,25.0
247984,23.0
247985,25.0
247986,29.0


In [ ]:
age_bins = pd.cut(valid_age, bins=[0,17,25,34,44,54,64,200],
                  labels=["0–17","18–25","26–34","35–44","45–54","55–64","65+"],
                  right=True, include_lowest=True)


In [ ]:
age_bins

,Vict Age
0,NaN
1,26–34
2,55–64
3,26–34
4,35–44
...,...
247983,18–25
247984,18–25
247985,18–25
247986,26–34


In [ ]:
victim_ages = age_bins.value_counts().reindex(
    ["0–17","18–25","26–34","35–44","45–54","55–64","65+"]
).fillna(0).astype(int)

In [ ]:
victim_ages

,count
Vict Age,
0–17,4528
18–25,28291
26–34,47470
35–44,42157
45–54,28353
55–64,20169
65+,14747


In [ ]:
pivot = df.pivot_table(index="AREA NAME", columns="hour", values="DR_NO", aggfunc="count", fill_value=0)
top_hours_by_area = pivot.idxmax(axis=1)

In [ ]:
pivot

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
AREA NAME,,,,,,,,,,,,,,,,,,,,,
77th Street,761,487,406,391,291,287,610,470,630,605,...,660,768,710,811,813,756,781,752,687,630
Central,703,698,496,377,341,334,439,570,632,714,...,874,919,912,975,1231,1119,1034,910,928,889
Devonshire,381,259,254,230,176,183,252,267,485,357,...,520,544,525,630,605,564,582,515,453,323
Foothill,369,217,211,195,178,155,245,234,312,300,...,367,383,410,449,434,381,424,338,396,319
Harbor,436,306,250,217,173,154,281,262,366,348,...,458,535,505,499,556,498,523,522,491,428
Hollenbeck,410,287,199,187,169,162,304,308,324,299,...,458,473,456,535,499,473,497,439,451,358
Hollywood,627,554,456,341,284,216,280,281,429,446,...,605,632,649,683,663,637,608,586,674,647
Mission,354,275,214,200,142,154,310,268,381,346,...,407,445,483,510,528,521,475,436,503,361
N Hollywood,564,380,307,288,283,258,314,286,435,402,...,575,622,603,627,722,600,624,509,588,470


In [ ]:
top_hours_by_area

,0
AREA NAME,
77th Street,12
Central,18
Devonshire,12
Foothill,12
Harbor,12
Hollenbeck,12
Hollywood,12
Mission,12
N Hollywood,12
